# 3D Tetrahedron Visualization with Quantum States

This notebook creates an interactive 3D visualization of a tetrahedron graph and its corresponding quantum state representation.
We use:
- `networkx` for graph creation
- `plotly` for interactive 3D visualization
- `qutip` for quantum state representation

In [1]:
# Initialize Jupyter widgets
import ipywidgets as widgets
from IPython.display import display, HTML

# Enable widget support
try:
    display(HTML('''<script>
        if (typeof Jupyter !== 'undefined') {
            require.config({
                paths: {
                    'jupyter-js-widgets': 'nbextensions/jupyter-js-widgets/extension'
                }
            });
            require(['jupyter-js-widgets'], function(widgets) {
                console.log('Widgets loaded successfully');
            });
        }
    </script>'''))
except:
    print('Note: Widget initialization code run. If widgets still don\'t display, please ensure you have the Jupyter extension installed in VSCode.')

In [3]:
import networkx as nx
import plotly.graph_objects as go
import numpy as np
import ipywidgets as widgets
from IPython.display import display, clear_output

# Create a tetrahedron graph
G = nx.tetrahedral_graph()

# Define node positions in 3D space
pos = {
    0: [0, 0, 0],
    1: [1, 0, 0],
    2: [0.5, 0.866, 0],
    3: [0.5, 0.289, 0.866]
}

# Extract node coordinates for plotting
node_x = [pos[node][0] for node in G.nodes()]
node_y = [pos[node][1] for node in G.nodes()]
node_z = [pos[node][2] for node in G.nodes()]

# Create edges coordinates for plotting
edge_x = []
edge_y = []
edge_z = []

for edge in G.edges():
    x0, y0, z0 = pos[edge[0]]
    x1, y1, z1 = pos[edge[1]]
    edge_x.extend([x0, x1, None])
    edge_y.extend([y0, y1, None])
    edge_z.extend([z0, z1, None])

# Create the 3D visualization
fig = go.Figure()

# Add edges (lines)
fig.add_trace(go.Scatter3d(
    x=edge_x, y=edge_y, z=edge_z,
    mode='lines',
    line=dict(color='blue', width=2),
    name='Edges'
))

# Add nodes (spheres)
fig.add_trace(go.Scatter3d(
    x=node_x, y=node_y, z=node_z,
    mode='markers+text',
    marker=dict(
        size=12,
        color='red',
    ),
    text=[str(i) for i in G.nodes()],
    name='Nodes'
))

# Update layout
fig.update_layout(
    title='Interactive 3D Tetrahedron',
    showlegend=True,
    scene=dict(
        xaxis_title='X',
        yaxis_title='Y',
        zaxis_title='Z',
        aspectmode='cube'
    ),
    width=400,
    height=400,
    margin=dict(t=50, b=0, l=0, r=0)
)

fig.show()

## Quantum State Representation

Now we'll create a quantum state corresponding to the tetrahedron structure where each edge hosts a spin-1/2 particle.
We'll use QuTiP to create this state as a tensor product of 6 spin-1/2 states (one for each edge).

In [ ]:
import qutip as qt
import numpy as np

# Create spin operators for each edge
# For spin-1/2, we use qubit operators
sx = qt.sigmax()  # Pauli X
sy = qt.sigmay()  # Pauli Y
sz = qt.sigmaz()  # Pauli Z

# Create basis states for spin up and down
up = qt.basis([2], 0)    # |0⟩ or |↑⟩
down = qt.basis([2], 1)   # |1⟩ or |↓⟩

# Create initial state with all spins up
# Tensor product of 6 up states (one for each edge)
psi = qt.tensor([up] * 6)

# Dictionary to map edges to their indices in the tensor product
edge_to_index = {tuple(sorted(edge)): idx for idx, edge in enumerate(G.edges())}

# Create individual spin operators for each edge
# These will be useful for creating observables and manipulating the state
spin_ops = {}
for edge, idx in edge_to_index.items():
    # Create identity operators for all positions
    op_list = [qt.qeye(2)] * 6
    # Replace the identity at idx with Pauli operators
    spin_ops[edge] = {
        'X': qt.tensor(op_list[:idx] + [sx] + op_list[idx+1:]),
        'Y': qt.tensor(op_list[:idx] + [sy] + op_list[idx+1:]),
        'Z': qt.tensor(op_list[:idx] + [sz] + op_list[idx+1:])
    }

# Example: measure Z component of spin on each edge
z_expectations = {edge: qt.expect(ops['Z'], psi) 
                for edge, ops in spin_ops.items()}

print('Initial state (all spins up):)')
print(psi)
print('\nZ-component expectations for each edge:')
for edge, val in z_expectations.items():
    print(f'Edge {edge}: {val:.3f}')

The code above creates:
1. A quantum state as a tensor product of 6 spin-1/2 states (one per edge)
2. Spin operators (X, Y, Z Pauli matrices) for each edge
3. A mapping between graph edges and their position in the tensor product
4. Initial measurements of the Z-component of spin on each edge

You can modify this state by applying operators to specific edges using the `spin_ops` dictionary.

## Three-Body Interactions on Faces

We'll now implement three-body interactions between the edges that form each face of the tetrahedron.
For each face, we create an interaction term between its three bounding edges.

In [5]:
# Define the faces of the tetrahedron
faces = [
    (0, 1, 2),  # vertices of face 1
    (0, 1, 3),  # vertices of face 2
    (0, 2, 3),  # vertices of face 3
    (1, 2, 3)   # vertices of face 4
]

# Function to get edges of a face
def get_face_edges(face_vertices):
    edges = []
    for i in range(3):
        edge = tuple(sorted([face_vertices[i], face_vertices[(i+1)%3]]))
        edges.append(edge)
    return edges

# Map faces to their edges
face_edges = {i: get_face_edges(face) for i, face in enumerate(faces)}

print('Edges for each face:')
for face_idx, edges in face_edges.items():
    print(f'Face {face_idx}: {edges}')

Edges for each face:
Face 0: [(0, 1), (1, 2), (0, 2)]
Face 1: [(0, 1), (1, 3), (0, 3)]
Face 2: [(0, 2), (2, 3), (0, 3)]
Face 3: [(1, 2), (2, 3), (1, 3)]


In [18]:
# Create three-body interaction Hamiltonian
def create_face_hamiltonian(face_idx, J=1.0):
    """Create interaction Hamiltonian for a single face"""
    edges = face_edges[face_idx]
    
    # Different coupling strengths for different faces
    J_values = {0: 1.0, 1: 0.5, 2: -0.7, 3: 0.3}
    J = J_values[face_idx]
    
    # Create interaction terms using Pauli operators
    H_face = 0
    for op in ['X', 'Y', 'Z']:  # Sum over all Pauli operator combinations
        term = J * spin_ops[edges[0]][op] * spin_ops[edges[1]][op] * spin_ops[edges[2]][op]
        H_face += term
    
    return H_face

# Create total Hamiltonian as sum of face Hamiltonians
H_total = sum(create_face_hamiltonian(face_idx) for face_idx in face_edges)

print('Total Hamiltonian created with different coupling strengths:')
print('Face 0: J = 1.0')
print('Face 1: J = 0.5')
print('Face 2: J = -0.7')
print('Face 3: J = 0.3')
print(f'Shape: {H_total.shape}')

Total Hamiltonian created with different coupling strengths:
Face 0: J = 1.0
Face 1: J = 0.5
Face 2: J = -0.7
Face 3: J = 0.3
Shape: (64, 64)


In [19]:
# Set up time evolution
times = np.linspace(0, 10, 100)  # Evolve from t=0 to t=10

# Evolve the state using QuTiP's sesolve
result = qt.sesolve(H_total, psi, times)

# Calculate expectation values of Z operators for each edge over time
z_expectations_t = {edge: [qt.expect(ops['Z'], state) 
                         for state in result.states]
                   for edge, ops in spin_ops.items()}

# Print the first few values for each edge to verify they're different
print("First few expectation values for each edge:")
for edge, expecs in z_expectations_t.items():
    print(f"Edge {edge}: {expecs[:5]}")

# Create interactive Plotly figure
fig = go.Figure()

# Add a trace for each edge's evolution
colors = ['blue', 'red', 'green', 'purple', 'orange', 'cyan']
for (edge, expecs), color in zip(z_expectations_t.items(), colors):
    fig.add_trace(go.Scatter(
        x=times,
        y=expecs,
        mode='lines',
        name=f'Edge {edge}',
        line=dict(color=color, width=2),
        hovertemplate='Time: %{x:.2f}<br>⟨σz⟩: %{y:.3f}<br>Edge: {edge}<extra></extra>'
    ))

# Update layout for better visualization
fig.update_layout(
    title='Evolution of Z-component of spin on each edge',
    xaxis_title='Time',
    yaxis_title='⟨σz⟩',
    hovermode='x unified',
    showlegend=True,
    width=600,
    height=400,
    template='plotly_white',
    legend=dict(
        yanchor='top',
        y=0.99,
        xanchor='right',
        x=0.99
    )
)

fig.show()

First few expectation values for each edge:
Edge (0, 1): [1.0, 0.9498989797533385, 0.8102309094596531, 0.6104786814299205, 0.3922630223027456]
Edge (0, 2): [1.0, 0.9405872030375467, 0.7784904941228163, 0.5576480875797946, 0.33758462257919625]
Edge (0, 3): [1.0, 0.9702088673372905, 0.8856379339841978, 0.7598491180673136, 0.612777906943434]
Edge (1, 2): [1.0, 0.9561238135807398, 0.8316267107193986, 0.6466113049147625, 0.4305140131686564]
Edge (1, 3): [1.0, 0.9862376232951225, 0.9462956379734349, 0.8840625277501099, 0.8055465880048482]
Edge (2, 3): [1.0, 0.9765675638926107, 0.9090793757205593, 0.8055647775289648, 0.6781378997370784]


This implementation:
1. Defines the faces of the tetrahedron in terms of their vertex indices
2. Maps each face to its three bounding edges
3. Creates a three-body interaction Hamiltonian for each face using Pauli operators
4. Combines the face Hamiltonians into a total Hamiltonian
5. Evolves the initial state (all spins up) using the Schrödinger equation
6. Visualizes the evolution of the Z-component of spin on each edge

The interaction term for each face couples all three edges bounding that face,
creating a genuinely three-body interaction rather than just pairwise couplings.

## Interactive Controls for Coupling Strengths

Use the sliders below to adjust the coupling strengths for each face of the tetrahedron.
The visualization will update automatically to show how different coupling strengths affect the evolution.

In [ ]:
# Create output widgets for plots
plot_output = widgets.Output()
controls_output = widgets.Output()

# Create sliders for each face's coupling strength
coupling_sliders = {}
for i in range(4):
    default_value = 1.0 if i == 0 else (0.5 if i == 1 else (-0.7 if i == 2 else 0.3))
    slider = widgets.FloatSlider(
        value=default_value,
        min=-2.0,
        max=2.0,
        step=0.1,
        description=f'Face {i}:',
        continuous_update=False,
        layout=widgets.Layout(width='400px')
    )
    coupling_sliders[i] = slider
    display(slider)

reset_button = widgets.Button(
    description='Reset Couplings',
    button_style='info',
    layout=widgets.Layout(width='200px')
)

def reset_couplings(b):
    default_values = {0: 1.0, 1: 1.0, 2: 1.0, 3: 1.0}
    for i, slider in coupling_sliders.items():
        slider.value = default_values[i]

reset_button.on_click(reset_couplings)
display(reset_button)

# Add some visual spacing
display(widgets.HTML("<br>"))

def update_visualization(change=None):
    # Update the 3D tetrahedron visualization with colored faces
    fig3d = go.Figure()
    
    # Add edges (lines)
    fig3d.add_trace(go.Scatter3d(
        x=edge_x, y=edge_y, z=edge_z,
        mode='lines',
        line=dict(color='blue', width=2),
        name='Edges'
    ))
    
    # Add vertices (nodes)
    fig3d.add_trace(go.Scatter3d(
        x=node_x, y=node_y, z=node_z,
        mode='markers+text',
        marker=dict(size=12, color='red'),
        text=[str(i) for i in G.nodes()],
        name='Vertices'
    ))
    
    # Add faces with colors based on coupling strengths
    for face_idx, face_verts in enumerate(faces):
        coupling = coupling_sliders[face_idx].value
        # Normalize color based on coupling strength
        color = f'rgb({int(255*(1+coupling/2))},{int(255*(1-abs(coupling)/2))},{int(255*(1-coupling/2))})'
        
        # Create triangular face
        x = [pos[v][0] for v in face_verts] + [pos[face_verts[0]][0]]
        y = [pos[v][1] for v in face_verts] + [pos[face_verts[0]][1]]
        z = [pos[v][2] for v in face_verts] + [pos[face_verts[0]][2]]
        
        fig3d.add_trace(go.Mesh3d(
            x=x, y=y, z=z,
            opacity=0.3,
            color=color,
            name=f'Face {face_idx} (J={coupling:.1f})'
        ))
    
    fig3d.update_layout(
        title='Tetrahedron with Coupling Strengths',
        scene=dict(
            xaxis_title='X',
            yaxis_title='Y',
            zaxis_title='Z',
            aspectmode='cube'
        ),
        width=500,
        height=500,
        margin=dict(t=50, b=0, l=0, r=0)
    )
    
    # Calculate new Hamiltonian and evolution
    def create_face_hamiltonian(face_idx):
        edges = face_edges[face_idx]
        J = coupling_sliders[face_idx].value
        H_face = 0
        for op in ['X', 'Y', 'Z']:
            term = J * spin_ops[edges[0]][op] * spin_ops[edges[1]][op] * spin_ops[edges[2]][op]
            H_face += term
        return H_face
    
    H_total = sum(create_face_hamiltonian(face_idx) for face_idx in face_edges)
    
    # Evolve the state
    result = qt.sesolve(H_total, psi, times)
    
    # Calculate expectation values
    z_expectations_t = {edge: [qt.expect(ops['Z'], state) 
                             for state in result.states]
                           for edge, ops in spin_ops.items()}
    
    # Update evolution plot
    fig_evolution = go.Figure()
    
    colors = ['blue', 'red', 'green', 'purple', 'orange', 'cyan']
    for (edge, expecs), color in zip(z_expectations_t.items(), colors):
        fig_evolution.add_trace(go.Scatter(
            x=times,
            y=expecs,
            mode='lines',
            name=f'Edge {edge}',
            line=dict(color=color, width=2),
            hovertemplate='Time: %{x:.2f}<br>⟨σz⟩: %{y:.3f}<br>Edge: {edge}<extra></extra>'
        ))
    
    fig_evolution.update_layout(
        title='Evolution of Z-component of spin on each edge',
        xaxis_title='Time',
        yaxis_title='⟨σz⟩',
        hovermode='x unified',
        showlegend=True,
        width=800,
        height=400,
        template='plotly_white'
    )
    
    # Display plots in fixed order
    clear_output(wait=True)
    
    # Redisplay controls
    for slider in coupling_sliders.values():
        display(slider)
    display(reset_button)
    display(widgets.HTML("<br>"))
    
    # Display plots
    display(fig_evolution)
    display(fig3d)

# Connect sliders to update function
for slider in coupling_sliders.values():
    slider.observe(update_visualization, names='value')

# Initial visualization
update_visualization()

FloatSlider(value=0.0, continuous_update=False, description='Face 0:', layout=Layout(width='400px'), max=2.0, …

FloatSlider(value=0.0, continuous_update=False, description='Face 1:', layout=Layout(width='400px'), max=2.0, …

FloatSlider(value=0.0, continuous_update=False, description='Face 2:', layout=Layout(width='400px'), max=2.0, …

FloatSlider(value=0.0, continuous_update=False, description='Face 3:', layout=Layout(width='400px'), max=2.0, …

Button(button_style='info', description='Reset Couplings', layout=Layout(width='200px'), style=ButtonStyle())

HTML(value='<br>')

# Mathematical Description of the Quantum Tetrahedron System

## 1. System Structure

Our system consists of a tetrahedron with:
- 4 vertices labeled $\{0,1,2,3\}$
- 6 edges, each carrying a spin-1/2 quantum state
- 4 faces, each bounded by 3 edges

## 2. Quantum State Space

Each edge carries a spin-1/2 state, so:
- Single edge Hilbert space: $\mathcal{H}_e \cong \mathbb{C}^2$
- Total Hilbert space: $\mathcal{H} = \bigotimes_{e=1}^6 \mathcal{H}_e \cong (\mathbb{C}^2)^{\otimes 6}$

The basis states for each edge are:
- Spin up: $|\uparrow\rangle = |0\rangle = \begin{pmatrix}1\\0\end{pmatrix}$
- Spin down: $|\downarrow\rangle = |1\rangle = \begin{pmatrix}0\\1\end{pmatrix}$

## 3. Spin Operators

For each edge $e$, we have the Pauli operators:

$$
\sigma^x_e = \begin{pmatrix}0 & 1\\1 & 0\end{pmatrix}, \quad
\sigma^y_e = \begin{pmatrix}0 & -i\\i & 0\end{pmatrix}, \quad
\sigma^z_e = \begin{pmatrix}1 & 0\\0 & -1\end{pmatrix}
$$

In the full Hilbert space, these become:

$\sigma^\alpha_e = I \otimes \cdots \otimes \sigma^\alpha \otimes \cdots \otimes I$

where $\sigma^\alpha$ appears in the $e$-th position ($\alpha = x,y,z$).

## 4. Three-Body Interactions

For each face $f$ of the tetrahedron, we have three edges $\{e_1^f, e_2^f, e_3^f\}$ forming its boundary. The interaction Hamiltonian for face $f$ is:

$$
H_f = J_f \sum_{\alpha=x,y,z} \sigma^\alpha_{e_1^f} \sigma^\alpha_{e_2^f} \sigma^\alpha_{e_3^f}
$$

where:
- $J_f$ is the coupling strength for face $f$ (controlled by sliders)
- The sum over $\alpha$ creates an isotropic interaction

## 5. Total Hamiltonian

The total Hamiltonian is the sum over all faces:

$$
H_{\text{total}} = \sum_{f=0}^3 H_f = \sum_{f=0}^3 J_f \sum_{\alpha=x,y,z} \sigma^\alpha_{e_1^f} \sigma^\alpha_{e_2^f} \sigma^\alpha_{e_3^f}
$$

## 6. Time Evolution

The system evolves according to the Schrödinger equation:

$$
i\hbar \frac{d}{dt}|\psi(t)\rangle = H_{\text{total}}|\psi(t)\rangle
$$

Given our initial state $|\psi(0)\rangle = |\uparrow\uparrow\uparrow\uparrow\uparrow\uparrow\rangle$, the solution is:

$$
|\psi(t)\rangle = e^{-iH_{\text{total}}t/\hbar}|\psi(0)\rangle
$$

We measure the z-component of spin on each edge as the system evolves:

$$
\langle\sigma^z_e(t)\rangle = \langle\psi(t)|\sigma^z_e|\psi(t)\rangle
$$

The plots show these expectation values as functions of time for each edge.